# ICESat-2 sea ice tutorial


## 	&#x2B50; Objectives
- Learn how to access/download ICESat-2 sea ice products (ATL07/ATL10/ATL20) via `icepyx` or `earthaccess` libraries.
- Examine what the ICESat-2 sea ice freeboard products (ATL10/ATL20) look like.
- Derive sea ice properties (sea ice thickness, floe size distribution, lead fraction) using ATL10 product.
- Map monthly sea ice freeboard using ATL20 product.

## &#x1F4CC; What is sea ice?
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Sea_ice_Drawing_General_features.svg/1920px-Sea_ice_Drawing_General_features.svg.png" alt="Sea ice" width = "600px">

Sea ice is frozen seawater that floats on the ocean surface. It forms in both the Arctic and the Antarctic in each hemisphere’s winter; it retreats in the summer, but does not completely disappear. Sea ice is not just a "flat" surface. Sea ice surface is very dynamic, so sea ice surface consists of various topograhic features, such as ridges (occur when sea ice floes collide) or leads (open water or thin ice space between sea ice floes). The fine resolution of ICESat-2 allows us to detect the details of these sea ice dynamic features.

## &#x1F4CC; How can laser altimeters measure sea ice freeboard/thickness?
<img src="https://www.mdpi.com/remotesensing/remotesensing-14-01069/article_deploy/html/images/remotesensing-14-01069-g004-550.jpg" alt="ICESat-2 data products" width="400px">

Sea ice freeboard means the height of sea ice surface above **local sea surface**. To calculate sea ice freeboard, it is necessary to subtract local sea surface height from the sea ice height.

$H_{f} = H_{obs} - H_{ssh}$

The **ATL07** product provides the surface height above the reference ellipsoid (WGS84). The ATL07 product aggregates 150 signal photons as a single "height segment", and the mean height of this segment is determined as the surface height. In general, each height segment has 10-20 m of length (strong beams).

Then, in order to calculate sea ice freeboard, it is necessary to separate the height signal from **open water (lead)** and **sea ice surface**. The ATL07 product has its own algorithm to distinguish leads from sea ice.

The **ATL10** product calculates the height difference between sea ice surface and lead surface. The local sea level ($H_{ssh}$) is calculated for every 10 km using detected leads.

## &#x1F4CC; ICESat-2 sea ice products
- <b> ATL07 </b> (sea ice surface heights): Surface height relative to WGS84 ellipsoid. Derived by aggregating 150 signal photons of the ATL03 product. 
- <b> ATL10 </b> (freeboard): Sea ice freeboard calculated from ATL07 product.
- <b> ATL20 </b> (daily/monthly gridded sea ice freeboard): 25-km gridded mean freeboard in daily/monthly scale.

<img src="https://nsidc.org/sites/default/files/images/Data/ICESat2_DataProducts_2023.png" alt="ICESat-2 data products" width="600px">

## &#x2705; Access & Download sea ice products (1) - Use icepyx

In this tutorial, first, we will use `icepyx` library to access and download ICESat-2 datasets.

In [ ]:
import icepyx as ipx

After importing `icepyx` library, we will create an query to request ICESat-2 data for our region/time of interest.

In [ ]:
# Data product: ATL10 (sea ice freeboard)
short_name = 'ATL10'

# Spatial extent: Ross Sea, Antarctica
spatial_extent = [-180, -78, -160, -74]

# Time ragne
date_range = ['2019-09-16','2019-09-16'] # first time period
# date_range = ['2019-11-13','2019-11-13'] # second time period

In [ ]:
reg=ipx.Query(short_name, spatial_extent, date_range,
              start_time = "12:00:00", endtime = "20:00:00")

In [ ]:
# Visualize the spatial extent
reg.visualize_spatial_extent()

In [ ]:
# Check the granule ids
gran_ids = reg.avail_granules(ids=True)
gran_ids

In [ ]:
print(reg.latest_version())
reg.product_summary_info()

In [ ]:
# Earthdata login information: Please put your own account information
reg.earthdata_login()

In [ ]:
# Order granules
reg.order_granules()

In [ ]:
# Download granules
path = "" # you can put your own directory where you want to download the files.
reg.download_granules(path)

## &#x2705; Access & Download sea ice products (2) - Use Earthaccess

Using icepyx, you can download ICESat-2 data for the date and region you are interested in. There is another way to access ICESat-2 data using another Python library `Earthaccess`. In the following Earthaccess example, you can access the raw ICESat-2 data remotely without downloading them to your local machine.

In [ ]:
# For searching NASA data
import earthaccess

In [ ]:
auth = earthaccess.login()

In [ ]:
Query = earthaccess.collection_query().keyword('ICESat-2')

In [ ]:
Query.hits()

In [ ]:
Query = earthaccess.collection_query().keyword('ATL10').cloud_hosted(True)
print(Query.hits())
collections = Query.fields(['ShortName', 'Version']).get(25)
print(collections)

In [ ]:
Query = earthaccess.granule_query().concept_id(
    'C2153574813-NSIDC_CPRD'
).temporal("2019-11-13 T10:00:00", "2019-11-13 T20:00:00"
).bounding_box(-180, -77, -160, -74)

# Bounding box: [West lon, South lat, East lon, North lat]
# ATL07: C2153574585-NSIDC_CPRD
# ATL10: C2153574813-NSIDC_CPRD
# ATL20: C2153577387-NSIDC_CPRD

In [ ]:
Query.hits()

In [ ]:
granules = Query.get(2)
[display(g) for g in granules];

In [ ]:
files = earthaccess.open(granules)
files

## &#x2705; Explore ATL10 sea ice freeboard product (single track)

Now, we will explore what the ATL10 sea ice freeboard data really looks like. Let's start with the two example tracks you downloaded from the previous parts.

### Import necessary libraries

In [ ]:
import h5py
import pyproj

# Ignore warning
import warnings
warnings.filterwarnings("ignore")

import cartopy.crs as ccrs

%matplotlib widget
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

### Read first track

In [ ]:
with h5py.File(filename,'r') as f:
    orient = f['orbit_info/sc_orient'][0]
    
orient

In [ ]:
# H5 file of the ATL10 track
filename = "processed_ATL10-02_20191113181045_07310501_005_02.h5"

# Create a list for saving ATL10 beam track data
track = []

with h5py.File(filename,'r') as f:
    
    # Check the orbit orientation
    orient = f['orbit_info/sc_orient'][0]
    
    if orient == 0:
        strong_beams = [f"gt{i}l" for i in [1, 2, 3]]
    elif orient == 1:
        strong_beams = [f"gt{i}l" for i in [1, 2, 3]]
    else:
        strong_beams = []

    for beam in strong_beams:
        
        lat = f[beam]['freeboard_beam_segment/beam_freeboard/latitude'][:]
        lon = f[beam]['freeboard_beam_segment/beam_freeboard/longitude'][:]
        seg_x = f[beam]['freeboard_beam_segment/beam_freeboard/seg_dist_x'][:] / 1000 # (m to km)
        seg_len = f[beam]['freeboard_beam_segment/height_segments/height_segment_length_seg'][:]
        fb = f[beam]['freeboard_beam_segment/beam_freeboard/beam_fb_height'][:]
        surface_type = f[beam]['freeboard_beam_segment/height_segments/height_segment_type'][:]
        fb[fb>100] = np.nan
        
        df = pd.DataFrame({'lat': lat, 'lon': lon, 'seg_x': seg_x, 'seg_len': seg_len,
                          'freeboard': fb, 'stype': surface_type})
        df['beam'] = beam
        df = df.dropna().reset_index(drop = True)
        
        track.append(df)

#### &#x1F50D; Which beam is strong/weak?

The relative location of strong and beak beams depends on the orientation of the satellite orbit. If the satellite moves in forward direction, right beams (i.e., GT1R, GT2R, GT3R) are STRONG beams. On the contrary, in the case of backward direction, left beams (i.e., GT1L, GT2L, GT3L) are WEAK beams.

<img src="https://agupubs.onlinelibrary.wiley.com/cms/asset/9e074a72-2eb9-4457-b6ad-5c57d5334ee5/jgrc23628-fig-0001-m.jpg" alt="Forward Backward" width="600px">


In [ ]:
# Check the orbit orientation
with h5py.File(filename,'r') as f:
    orient = f['orbit_info/sc_orient'][0]

# 0: backward; 1: forward; 2: transition (do not use)
print(orient)

### Draw freeboard histograms

In [ ]:
plt.figure(figsize = (8, 4))
for df in track:
    
    plt.hist(df['freeboard'], bins = 100, range = (0,2), label = df.beam[0], histtype = 'step')
    print(f"Beam {df.beam[0]}")
    print(f"   - mean freeboard: {df.freeboard.mean():.3f} m")
    print(f"   - std. dev. freeboard: {df.freeboard.std():.3f} m")
    
plt.xticks(np.arange(0,2,0.1))
plt.xlim(0,2)
plt.grid(ls = ":", lw = 0.5)
plt.legend()
plt.xlabel("Freeboard (m)")
plt.ylabel("Count")
plt.show()

### Draw freeboard track

In [ ]:
# Select which beam you want to explore (0, 1, 2)
df = track[0]

plt.figure(figsize = (8, 3))

# Masking for normal sea ice
mask_ice = (df.stype == 1)

# Masking for specular lead
mask_specular = (df.stype >= 2) & (df.stype <= 5)

# Masking for dark lead
mask_dark = (df.stype >= 6) & (df.stype <= 9)

plt.scatter(df.seg_x[mask_ice], df.freeboard[mask_ice], s = 2, color = "c", label = "Non-lead")
plt.scatter(df.seg_x[mask_specular], df.freeboard[mask_specular], s = 5, color = "r", label = "Specular lead")
plt.scatter(df.seg_x[mask_dark], df.freeboard[mask_dark], s = 5, color = "k", label = "Dark lead")
plt.axhline(0.15, color = "b", ls = "--")
plt.xlabel("Along-track distance (km)")
plt.ylabel("Freeboard (m)")
plt.legend()

# plt.xlim(32890, 32920)
plt.show()

In [ ]:
## Calculate lead fraction
print("Sea ice (%): ", np.sum(df.seg_len[mask_ice]) / np.sum(df.seg_len) * 100)
print("Specular lead (%): ", np.sum(df.seg_len[mask_specular]) / np.sum(df.seg_len) * 100)
print("Dark lead (%): ", np.sum(df.seg_len[mask_dark]) / np.sum(df.seg_len) * 100)
print("Freeboard < 0.15 m (%): ", np.sum(df.seg_len[df.freeboard < 0.15]) / np.sum(df.seg_len) * 100)

## &#x1F4A1; Do it yourself!
Let's try the same steps for another track data for the Ross Sea Sea region. The first track we just examined is from September, but this second track is from November. Compare how the freeboard and lead fraction has been changed for these two months.

In [ ]:
filename = "processed_ATL10-02_20191113181045_07310501_005_02.h5"

In [ ]:
# Plot lead fraction, floe size distribution, mean freeboard, etc.

## &#x2705; Explore sea ice freeboard products (gridded product)

Now we will explore gridded sea ice freeboard product (ATL20).

### Antarctic example

We will visualize the Antractic sea ice in September 2021. In general, the Antarctic sea ice extent reaches maximum in September, so you will be able to see the maximum sea ice extent.

In [ ]:
# Query Antarctic ATL20 data using earthaccess (September)
Query = earthaccess.granule_query().concept_id(
    'C2153577387-NSIDC_CPRD'
).temporal("2021-09-02", "2021-09-30"
).bounding_box(-180, -90, 180, -60)

granules = Query.get()
granules

In [ ]:
# Open file
files = earthaccess.open(granules)
files[0]

In [ ]:
# Read h5 file
with h5py.File(files[0],'r') as f:
    mean_fb = f['monthly']['mean_fb'][:]
    mean_fb[mean_fb > 1000] = np.nan
    
    # ATL20 is 25 km grid data
    lat = f['grid_lat'][:] # Latitude
    lon = f['grid_lon'][:] # Longitude
    
    # Spatial reference: NSIDC Sea Ice Polar Stereographic South (EPSG:3412)
    x = f['grid_x'][:]
    y = f['grid_y'][:]
    xx, yy = np.meshgrid(x, y)

In [ ]:
fig=plt.figure(figsize=(6, 6))

# Use the in-built northpolarstereo to visualize (true scale latitude is from NSIDC polar stereographic)
ax = plt.axes(projection = ccrs.SouthPolarStereo(central_longitude=0, true_scale_latitude = -70))

pm  = ax.pcolormesh(xx ,yy, mean_fb, vmin = 0, vmax = 0.8)
ax.coastlines('10m', linewidth = 0.5) # add coastline
# Add gridline
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='lightgrey', alpha=0.5, linestyle='--')

clb = plt.colorbar(pm, ax = ax, shrink = 0.4)
clb.set_label('Sea ice freeboard (m)', rotation = 270, fontsize = 11, va = 'bottom')

### Arctic example

For the Arctic, we will visualize sea ice freeboard map in March 2021.

In [ ]:
Query = earthaccess.granule_query().concept_id(
    'C2153577387-NSIDC_CPRD'
).temporal("2021-03-02", "2021-03-30"
).bounding_box(-180, 60, 180, 90)

granules = Query.get()

files = earthaccess.open(granules)

with h5py.File(files[0],'r') as f:
    mean_fb = f['monthly']['mean_fb'][:]
    mean_fb[mean_fb > 1000] = np.nan

    lat = f['grid_lat'][:]
    lon = f['grid_lon'][:]
    
    # Spatial reference: NSIDC Sea Ice Polar Stereographic North (EPSG:3411)
    x = f['grid_x'][:]
    y = f['grid_y'][:]
    xx, yy = np.meshgrid(x, y)

In [ ]:
fig=plt.figure(figsize=(6, 6))
# Use the in-built northpolarstereo to visualize (true scale latitude is from NSIDC polar stereographic)
ax = plt.axes(projection = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude = 70))

pm  = ax.pcolormesh(xx ,yy, mean_fb, vmin = 0, vmax = 0.8)
ax.coastlines('10m', linewidth = 0.5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='lightgrey', alpha=0.5, linestyle='--')
clb = plt.colorbar(pm, ax = ax, shrink = 0.4)
clb.set_label('Sea ice freeboard (m)', rotation = 270, fontsize = 11, va = 'bottom')

## &#x1F4A1; Do it yourself!
Let's visualize the griddged freeboard for another month. 

#### Credited by Younghyun Koo (kooala317@gmail.com)